# imports and setup

In [14]:
import time
import os
import json
import glob
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

In [16]:
# Define scopes for accessing Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']
MEMORY_FILE = 'processed_files.json'
CREDENTIALS_FILE_WILDCARDED = './.credentials/client_secret*.json'
TOKEN_FILE = './.credentials/token.json'

# helper functions

In [17]:
def find_file_with_wildcard(pattern):
    """Find file with wildcard pattern, raise error if no match or multiple matches."""
    # Use glob to find matching files
    matching_files = glob.glob(pattern)
    
    # Check if no file was found
    if len(matching_files) == 0:
        raise FileNotFoundError(f"No files found matching pattern: {pattern}")
    
    # Check if more than one file was found
    if len(matching_files) > 1:
        raise ValueError(f"Multiple files found matching pattern: {pattern}")
    
    # Return the matched file path
    return matching_files[0]

CREDENTIALS_FILE = find_file_with_wildcard(CREDENTIALS_FILE_WILDCARDED)

def authenticate_google_drive():
    """Authenticate with Google Drive and return the service object."""
    creds = None
    # Check if token.json exists (token storage for authenticated user)
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
    
    # Authenticate if credentials are not valid or do not exist
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save credentials for the next run
        with open(TOKEN_FILE, 'w') as token:
            token.write(creds.to_json())
    
    # Build the Drive API service
    return build('drive', 'v3', credentials=creds)

def list_files_by_name(service, file_name):
    """List files with a specific name from Google Drive."""
    try:
        results = service.files().list(
            q=f"name = '{file_name}'",
            fields="nextPageToken, files(id, name, mimeType, modifiedTime, parents)",
            orderBy="modifiedTime desc"
        ).execute()
        return results.get('files', [])
    except HttpError as error:
        print(f"An error occurred: {error}")
        return []

def process_file(file):
    """Placeholder function to process a file."""
    print(f"Processing file: {file['name']} (ID: {file['id']})")

def move_file_to_folder(service, file_id, folder_id):
    """Move a file to a specific folder in Google Drive."""
    try:
        # Retrieve the current parents
        file = service.files().get(fileId=file_id, fields='parents').execute()
        previous_parents = ",".join(file.get('parents', []))

        # Move the file to the new folder
        service.files().update(
            fileId=file_id,
            addParents=folder_id,
            removeParents=previous_parents,
            fields='id, parents'
        ).execute()
        print(f"File {file_id} has been moved to folder {folder_id}.")
    except HttpError as error:
        print(f"An error occurred while moving the file: {error}")

def load_processed_files():
    """Load the list of processed file IDs from a local memory file."""
    if os.path.exists(MEMORY_FILE):
        with open(MEMORY_FILE, 'r') as file:
            return set(json.load(file))
    return set()

def save_processed_files(processed_files):
    """Save the list of processed file IDs to a local memory file."""
    with open(MEMORY_FILE, 'w') as file:
        json.dump(list(processed_files), file)

def monitor_google_drive(target_folder_id, interval=60):
    """Periodically check for new files, process them, and move them."""
    service = authenticate_google_drive()
    processed_files = load_processed_files()

    while True:
        print("Checking for new files named 'pleco_flashcards.xml'...")
        files = list_files_by_name(service, "pleco_flashcards.xml")

        for file in files:
            if file['id'] not in processed_files:
                # Process the file
                process_file(file)

                # Move the file to the specified folder
                move_file_to_folder(service, file['id'], target_folder_id)

                # Mark the file as processed and save to memory
                processed_files.add(file['id'])
                save_processed_files(processed_files)
        
        # Wait for the specified interval before checking again
        time.sleep(interval)


# sandbox

In [19]:
service = authenticate_google_drive()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=624862442234-4iconv0m99pu1luia7lub9c8m7t8of0b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52451%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=NXRN6854KN3tFMitYhiveaEPzJRnbE&access_type=offline


In [23]:
results = service.files().list(
    q="'root' in parents",  # Only get items in the root directory
    fields="nextPageToken, files(id, name, mimeType, parents)"
).execute()

In [24]:

items = results.get('files', [])

In [25]:
items

[{'mimeType': 'application/xml',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '18FY69M7X4WbjrsE1J1bLtdgNW4BtLUWl',
  'name': 'pleco_flashcards.xml'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '1S_r1Y_uGNmwlYmqByyjVa1pnQVC07Plf',
  'name': '_Pleco'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '1sZqS9t5sToDIKTYKFa6MmUgX6DR912K8',
  'name': 'Games'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '18kihcLPaWxxKcH2pwGdmNKpUuCj4gEtB',
  'name': 'Archive'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '1rX5qfrIpHpu-2ssPRrBNrRvJJPoyThyE',
  'name': 'Music'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'parents': ['0ANOuDrZlca71Uk9PVA'],
  'id': '1Gsw6_vNfbH4qTdvEg9ps91ygulCLnVBf',
  'name': 'Anki'},
 {'mimeType': 'application/vnd.google-apps.folder',
  'parents': ['0ANOu

In [ ]:

if __name__ == "__main__":
    # Replace 'your_target_folder_id' with the actual folder ID where files should be moved
    target_folder_id = 'your_target_folder_id'
    monitor_google_drive(target_folder_id, interval=300)  # Check every 5 minutes
